In [2]:
import mat73
import pandas as pd
import numpy as np
import scipy.io
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler

In [3]:
import os

DATA_DIR = ''
if 'google.colab' not in str(get_ipython()):
    if "anuja" in os.environ.get('USER'):
        DATA_DIR = 'data/'
    elif 'ubuntu' in os.environ.get('USER'):
        DATA_DIR = '/home/ubuntu/Martyna/repo/AI4Health/DATAfoof/'
    

## Sensor Level

In [4]:
foof = pd.read_csv(DATA_DIR+"/sensor_intercept_slope.csv")
columns = [['Electrode %d - Intercept'%(i), 'Electrode %d - Slope'%(i)] for i in range(1, 106)]
columns = list(np.array(columns).flatten())
columns.append('IDs')
foof.columns = columns
foof

,Electrode 1 - Intercept,Electrode 1 - Slope,Electrode 2 - Intercept,Electrode 2 - Slope,Electrode 3 - Intercept,Electrode 3 - Slope,Electrode 4 - Intercept,Electrode 4 - Slope,Electrode 5 - Intercept,Electrode 5 - Slope,...,Electrode 101 - Slope,Electrode 102 - Intercept,Electrode 102 - Slope,Electrode 103 - Intercept,Electrode 103 - Slope,Electrode 104 - Intercept,Electrode 104 - Slope,Electrode 105 - Intercept,Electrode 105 - Slope,IDs
0,1.610769,1.793923,1.639958,1.803579,1.622763,1.867163,1.596286,1.868653,1.459489,1.880624,...,1.637747,1.472414,1.775967,1.534612,1.803604,1.575800,1.869977,0.986272,1.825774,NDARAA075AMK
1,1.572269,1.762184,1.615888,1.938907,1.515331,1.878222,1.577389,1.888830,1.645799,1.892993,...,1.698141,1.698537,1.854421,1.486611,1.755794,1.517099,1.845983,1.486650,1.888544,NDARAA112DMH
2,1.414507,1.818785,1.504383,1.902537,1.563395,1.936381,1.554663,2.085351,1.566065,2.106747,...,1.847331,1.417519,1.869143,1.323095,1.838333,1.496008,1.895424,1.593155,2.095749,NDARAA117NEJ
3,0.561062,1.068994,0.333554,0.991180,0.555779,1.304169,0.665864,1.491464,0.524828,1.573271,...,1.464311,0.576556,1.265274,0.199181,0.863242,0.246682,1.010393,0.703331,1.724831,NDARAA947ZG5
4,1.262007,1.901401,1.305927,1.924721,1.293914,1.924840,1.226456,1.947274,0.818595,1.905113,...,1.859064,1.158560,1.863190,1.173287,1.773532,1.252168,1.878925,0.918020,1.749441,NDARAA948VFH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2037,0.648706,1.508826,0.591809,1.640814,0.640782,1.665891,0.455759,1.679019,0.866016,1.786340,...,1.536850,0.528776,1.478037,0.466827,1.767683,0.570274,1.664126,1.351549,1.996940,NDARZN277NR6
2038,1.332278,1.729382,1.387863,1.788208,1.452167,1.841329,1.524229,1.976239,1.480653,2.022584,...,1.876485,1.214977,1.795080,1.456392,1.835807,1.399127,1.795906,1.380795,2.036327,NDARZN578YDP
2039,0.746123,1.324138,0.653335,1.285734,0.564490,1.271931,0.375903,1.245331,0.258508,1.206112,...,1.307891,0.691917,1.310004,0.604437,1.283587,0.564744,1.273098,0.339229,1.050644,NDARZN610GTY
2040,0.976055,1.441830,0.823197,1.407064,0.879368,1.464799,0.902699,1.512205,1.012450,1.505105,...,1.341166,0.598687,1.282644,0.697733,1.421177,0.753861,1.457204,0.781225,1.470061,NDARZN677EYE


In [5]:
data = scipy.io.loadmat(DATA_DIR+'x.mat')  
df = pd.DataFrame(data['x'].reshape((data['x'].shape[0], -1)))

# sparsing
# df = np.array(df).reshape(data['x'].shape)
# df_sparsed = np.concatenate([np.expand_dims(df[:,:,i:i+2].mean(axis = 2), axis = 2) for i in range(0, data['x'].shape[2]-2, 2)], axis = 2)
# df = pd.DataFrame(df_sparsed.reshape((df_sparsed.shape[0], -1)))
# print(df_sparsed.shape)

# #scaling
norm = MinMaxScaler().fit(df)
df = norm.transform(df)
df = pd.DataFrame(df.reshape((df.shape[0], -1)))

columns = np.asarray([['Electrode %d - %.1f Hz'%(i+1, j)] for i in range(105) for j in np.arange(1, 40.5, 0.5)])
df.columns = columns
df['IDs'] = foof['IDs']
df

,"(Electrode 1 - 1.0 Hz,)","(Electrode 1 - 1.5 Hz,)","(Electrode 1 - 2.0 Hz,)","(Electrode 1 - 2.5 Hz,)","(Electrode 1 - 3.0 Hz,)","(Electrode 1 - 3.5 Hz,)","(Electrode 1 - 4.0 Hz,)","(Electrode 1 - 4.5 Hz,)","(Electrode 1 - 5.0 Hz,)","(Electrode 1 - 5.5 Hz,)",...,"(Electrode 105 - 36.0 Hz,)","(Electrode 105 - 36.5 Hz,)","(Electrode 105 - 37.0 Hz,)","(Electrode 105 - 37.5 Hz,)","(Electrode 105 - 38.0 Hz,)","(Electrode 105 - 38.5 Hz,)","(Electrode 105 - 39.0 Hz,)","(Electrode 105 - 39.5 Hz,)","(Electrode 105 - 40.0 Hz,)",IDs
0,1.165485e-11,2.269649e-10,3.808305e-09,5.353586e-08,7.565996e-07,0.000011,0.000165,0.001702,0.011524,0.050167,...,4.648661e-09,1.879936e-09,7.239990e-10,2.777809e-10,1.068953e-10,4.127468e-11,1.599129e-11,6.216467e-12,2.424931e-12,NDARAA075AMK
1,5.743954e-02,2.460472e-01,4.457226e-01,3.251448e-01,1.054350e-01,0.044332,0.159810,0.238870,0.150660,0.181196,...,9.069666e-06,2.853855e-06,8.762598e-07,2.795727e-07,9.396118e-08,3.309113e-08,1.205145e-08,4.475759e-09,1.676995e-09,NDARAA112DMH
2,3.181883e-06,6.452939e-05,8.682196e-04,7.378562e-03,4.284223e-02,0.143420,0.305064,0.375769,0.318504,0.217702,...,2.070398e-07,8.101691e-08,2.998807e-08,1.098404e-08,4.008093e-09,1.457652e-09,5.283416e-10,1.908627e-10,6.871824e-11,NDARAA117NEJ
3,2.559403e-01,3.033666e-01,3.496544e-01,3.730970e-01,3.986542e-01,0.360040,0.306996,0.256529,0.263354,0.201815,...,1.293754e-01,1.192672e-01,1.062752e-01,9.575781e-02,8.783538e-02,8.205319e-02,7.806514e-02,7.564052e-02,7.464273e-02,NDARAA947ZG5
4,1.259914e-05,2.329821e-04,2.775527e-03,1.994995e-02,9.327162e-02,0.239264,0.371663,0.320721,0.188774,0.104396,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NDARAA948VFH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2037,1.246152e-05,4.409610e-04,7.140223e-03,5.037035e-02,1.674477e-01,0.221385,0.128560,0.031509,0.010560,0.023736,...,9.096372e-03,6.551053e-03,4.539767e-03,3.166834e-03,2.238763e-03,1.604574e-03,1.165964e-03,8.589819e-04,6.415888e-04,NDARZN277NR6
2038,1.226810e-02,1.509323e-02,1.904249e-02,2.345693e-02,3.051433e-02,0.035372,0.040242,0.040813,0.042936,0.048525,...,9.100567e-04,6.045304e-04,3.849341e-04,2.457844e-04,1.584299e-04,1.031353e-04,6.780565e-05,4.502042e-05,3.018812e-05,NDARZN578YDP
2039,8.275256e-03,1.323160e-02,2.506360e-02,5.001811e-02,1.031895e-01,0.173438,0.252636,0.285581,0.291097,0.280583,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NDARZN610GTY
2040,4.019794e-03,8.629347e-03,1.815964e-02,3.566658e-02,7.071627e-02,0.119430,0.189233,0.255499,0.342001,0.469486,...,5.714034e-04,2.959543e-04,1.450661e-04,7.039729e-05,3.404998e-05,1.642201e-05,7.897488e-06,3.787084e-06,1.810816e-06,NDARZN677EYE


In [6]:
beh = pd.read_csv(DATA_DIR+"behaviorals.csv")
print('Before:'+str(beh.shape))

most_common_disorders = ['Attention-Deficit/Hyperactivity Disorder', 'Anxiety Disorders', 'Specific Learning Disorder',
                         'Autism Spectrum Disorder', 'Disruptive', 'Communication Disorder',
                         'Depressive Disorders',  'No Diagnosis Given', 'Other Disorders']

category_columns = ['DX_' + str(i).zfill(2) + '_Cat' for i in range(1, 11)] +\
                   ['DX_' + str(i).zfill(2) + '_Sub' for i in range(1, 11)]

# removing patients with incomplete eval
initial_size = beh.shape[0]
beh = beh[beh.DX_01 != 'No Diagnosis Given: Incomplete Eval']
beh = beh.reset_index(drop=True)
new_size = beh.shape[0]

print('After:'+str(beh.shape))
print('Removing', initial_size - new_size,
      'patients as their evaluations was incomplete.')

Before:(3076, 177)
After:(2939, 177)
Removing 137 patients as their evaluations was incomplete.


In [7]:
no_diagnosis_given = 'No Diagnosis Given'
diagnoses_to_ids = {disorder: i for i, disorder in enumerate(most_common_disorders)}
diagnoses_to_ids

{'Attention-Deficit/Hyperactivity Disorder': 0,
 'Anxiety Disorders': 1,
 'Specific Learning Disorder': 2,
 'Autism Spectrum Disorder': 3,
 'Disruptive': 4,
 'Communication Disorder': 5,
 'Depressive Disorders': 6,
 'No Diagnosis Given': 7,
 'Other Disorders': 8}

In [8]:
def get_disorder(data, row, index):
    disorder = data.iloc[row][category_columns[index]]

    if disorder == 'Neurodevelopmental Disorders':
        disorder = data.iloc[row][category_columns[index + 10]]

    return disorder

order_of_disorders = []
for k in range(beh.shape[0]):
    i = 0
    disorder = get_disorder(beh, k, i)
    disorders_patient = []
    while not pd.isnull(disorder):
        if disorder in diagnoses_to_ids:
            if diagnoses_to_ids[disorder] not in disorders_patient:
                disorders_patient.append(diagnoses_to_ids[disorder])
        else:
            if diagnoses_to_ids['Other Disorders'] not in disorders_patient:
                disorders_patient.append(diagnoses_to_ids['Other Disorders'])
        i += 1
        if i == len(category_columns):
            break
        disorder = get_disorder(beh, k, i)

        
    order_of_disorders.append(disorders_patient)

In [9]:
other_disorders = []
no_diagnosis_given = []
for i in order_of_disorders:
    if 7 in i:
        no_diagnosis_given.append(1)
        i.remove(7)
    else:
        no_diagnosis_given.append(0)
    if 8 in i:
        other_disorders.append(1)
        i.remove(8)
    else:
        other_disorders.append(0)

In [10]:
max_len_order = np.max([len(x) for x in order_of_disorders])

# pad with a new token denoting the pad token
pad_token = len(most_common_disorders)
bod_token = len(most_common_disorders) + 1
eod_token = len(most_common_disorders) + 2

order_of_disorders = [[bod_token] + x + [eod_token] + [pad_token] * (max_len_order - len(x)) for x in order_of_disorders]

order_of_disorders = np.array(order_of_disorders)

classes = np.zeros((len(most_common_disorders),
                    beh.shape[0]), dtype=np.int32)

df_disorders = beh[category_columns]

for i, disorder in enumerate(most_common_disorders):
    mask = df_disorders.select_dtypes(include=[object]). \
        applymap(lambda x: disorder in x if pd.notnull(x) else False)

    disorder_df = df_disorders[mask.any(axis=1)]

    np.add.at(classes[i], disorder_df.index.values, 1)

behaviour_data_columns = beh.columns.values.astype(np.str)

columns_to_drop = behaviour_data_columns[
    np.flatnonzero(np.core.defchararray.find(behaviour_data_columns, 'DX') != -1)]

behaviour_data = beh.drop(columns=columns_to_drop)

for disorder, classification in zip(most_common_disorders, classes):
    behaviour_data[disorder] = classification

behaviour_data['order_diagnoses'] = list(order_of_disorders)

In [11]:
common_disorders = ['Attention-Deficit/Hyperactivity Disorder', 'Anxiety Disorders', 'Specific Learning Disorder',
                         'Autism Spectrum Disorder', 'Disruptive', 'Communication Disorder',
                         'Depressive Disorders']

labels=behaviour_data[["IDs"]+list(common_disorders)]
labels["Other Disorders"] = other_disorders
labels

/tmp/ipykernel_2331178/226828375.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels["Other Disorders"] = other_disorders


,IDs,Attention-Deficit/Hyperactivity Disorder,Anxiety Disorders,Specific Learning Disorder,Autism Spectrum Disorder,Disruptive,Communication Disorder,Depressive Disorders,Other Disorders
0,NDARAA075AMK,0,0,0,0,0,0,0,0
1,NDARAA112DMH,1,0,0,0,1,0,0,1
2,NDARAA117NEJ,1,0,0,0,1,0,0,1
3,NDARAA306NT2,1,1,1,0,0,1,0,1
4,NDARAA504CRN,1,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
2934,NDARZZ007YMP,0,0,0,1,0,0,0,1
2935,NDARZZ740MLM,1,0,0,0,0,0,0,0
2936,NDARZZ810LVF,0,0,0,1,0,1,0,1
2937,NDARZZ830JM7,0,0,0,1,0,0,0,1


In [12]:
add_features = behaviour_data[['Sex', 'Age','IDs']]
add_features.shape

(2939, 3)

In [13]:
site = pd.read_csv(DATA_DIR+"dataAfterMarch.csv")
site.columns

Index(['Unnamed: 0', 'IDs', 'Age', 'Release_Number', 'Sex', 'Study_Site',
       'SWAN_Total', 'DX_01', 'DX_01_ByHx', 'DX_01_Cat',
       ...
       'DX_10_PRem', 'DX_10_Past_Doc', 'DX_10_Presum', 'DX_10_RC', 'DX_10_Rem',
       'DX_10_RuleOut', 'DX_10_Spec', 'DX_10_Sub', 'DX_10_Time', 'NoDX'],
      dtype='object', length=158)

In [14]:
site = site[['Study_Site','IDs']]

In [15]:
df = pd.merge(df, foof, on='IDs', how='inner')
df = pd.merge(df, labels, on='IDs', how='inner')
df = pd.merge(df, add_features, on='IDs', how='inner')
df = pd.merge(df, site, on='IDs', how='inner')
df

,"(Electrode 1 - 1.0 Hz,)","(Electrode 1 - 1.5 Hz,)","(Electrode 1 - 2.0 Hz,)","(Electrode 1 - 2.5 Hz,)","(Electrode 1 - 3.0 Hz,)","(Electrode 1 - 3.5 Hz,)","(Electrode 1 - 4.0 Hz,)","(Electrode 1 - 4.5 Hz,)","(Electrode 1 - 5.0 Hz,)","(Electrode 1 - 5.5 Hz,)",...,Anxiety Disorders,Specific Learning Disorder,Autism Spectrum Disorder,Disruptive,Communication Disorder,Depressive Disorders,Other Disorders,Sex,Age,Study_Site
0,1.165485e-11,2.269649e-10,3.808305e-09,5.353586e-08,7.565996e-07,0.000011,0.000165,0.001702,0.011524,0.050167,...,0,0,0,0,0,0,0,1,6.728040,2
1,5.743954e-02,2.460472e-01,4.457226e-01,3.251448e-01,1.054350e-01,0.044332,0.159810,0.238870,0.150660,0.181196,...,0,0,0,1,0,0,1,0,5.545744,1
2,3.181883e-06,6.452939e-05,8.682196e-04,7.378562e-03,4.284223e-02,0.143420,0.305064,0.375769,0.318504,0.217702,...,0,0,0,1,0,0,1,0,7.475929,1
3,2.559403e-01,3.033666e-01,3.496544e-01,3.730970e-01,3.986542e-01,0.360040,0.306996,0.256529,0.263354,0.201815,...,0,1,1,0,0,0,1,0,13.627880,1
4,1.259914e-05,2.329821e-04,2.775527e-03,1.994995e-02,9.327162e-02,0.239264,0.371663,0.320721,0.188774,0.104396,...,0,0,0,0,0,0,1,1,7.982660,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1916,5.137044e-06,2.553540e-05,1.165434e-04,4.649161e-04,1.753336e-03,0.005275,0.013941,0.029402,0.057570,0.108259,...,1,0,0,0,0,0,0,1,11.629363,3
1917,1.246152e-05,4.409610e-04,7.140223e-03,5.037035e-02,1.674477e-01,0.221385,0.128560,0.031509,0.010560,0.023736,...,1,0,0,0,0,1,0,1,14.878736,1
1918,8.275256e-03,1.323160e-02,2.506360e-02,5.001811e-02,1.031895e-01,0.173438,0.252636,0.285581,0.291097,0.280583,...,0,0,0,0,0,0,1,1,16.379534,1
1919,4.019794e-03,8.629347e-03,1.815964e-02,3.566658e-02,7.071627e-02,0.119430,0.189233,0.255499,0.342001,0.469486,...,0,0,0,0,0,0,0,1,15.029545,3


In [16]:
df.to_csv('out.csv')  

In [17]:
out = pd.read_csv(DATA_DIR+"out.csv")

In [18]:
out.columns[-11:]

Index(['Attention-Deficit/Hyperactivity Disorder', 'Anxiety Disorders',
       'Specific Learning Disorder', 'Autism Spectrum Disorder', 'Disruptive',
       'Communication Disorder', 'Depressive Disorders', 'Other Disorders',
       'Sex', 'Age', 'Study_Site'],
      dtype='object')

In [19]:
rest = out[['Attention-Deficit/Hyperactivity Disorder', 'Anxiety Disorders',
       'Specific Learning Disorder', 'Autism Spectrum Disorder', 'Disruptive',
       'Communication Disorder', 'Depressive Disorders', 'Other Disorders',
       'Sex', 'Age', 'Study_Site', 'IDs']]

In [20]:
xtrain = out.drop('IDs', axis = 1)

In [21]:
xtrain = np.asarray(xtrain)

In [22]:
xtrain = xtrain[:,:-11]

In [23]:


import scipy.io
import mat73
import pandas as pd
import numpy as np
from numpy import array
from numpy.random import uniform
from numpy import hstack
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, LeakyReLU, BatchNormalization
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.svm import SVC as svm
from sklearn.linear_model import LogisticRegression as lg
from sklearn.metrics import confusion_matrix,accuracy_score,balanced_accuracy_score,f1_score
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.exceptions import ConvergenceWarning
from warnings import simplefilter,filterwarnings

# ignore all future warnings1
simplefilter(action='ignore', category=FutureWarning)



In [24]:

norm = preprocessing.MinMaxScaler().fit(xtrain)


In [25]:
# transform training data
xtrain = norm.transform(xtrain)
print(xtrain.shape)

print('Applying PCA...')
pca = PCA(.90) # 95% variance retained
pca.fit(xtrain)

# transform data
xtrain = pca.transform(xtrain)
print(xtrain.shape)

(1921, 8506)
Applying PCA...
(1921, 111)


In [26]:
df = pd.DataFrame(xtrain)

In [27]:
df

,0,1,2,3,4,5,6,7,8,9,...,101,102,103,104,105,106,107,108,109,110
0,-3.135464,3.094747,-3.792136,1.046076,-2.982556,-0.760645,-1.073833,-1.305185,0.227560,0.439760,...,0.081144,0.272121,-0.067619,0.194377,-0.237910,0.332506,0.149940,0.451322,-0.478881,-0.067975
1,-8.735916,-4.395335,5.553377,3.116078,1.681089,-0.786336,-0.584752,-1.174209,1.274068,0.228281,...,-0.997638,0.045865,0.002761,-0.109978,-0.617442,0.312267,-0.343493,0.188629,0.095878,-0.460363
2,1.701273,4.359496,-1.785404,0.665948,1.634100,-0.659093,0.714908,-1.606840,2.279333,0.542853,...,0.206803,0.231394,-0.168300,0.023363,-0.170938,0.450082,-0.175064,-0.347301,0.173784,0.311482
3,-8.155979,-5.996179,6.760203,2.807360,2.340080,1.566508,1.399411,0.363007,-1.309300,1.665582,...,0.016391,0.524637,-0.868596,-0.610392,-0.715227,-0.111242,1.916105,0.294494,-0.229206,0.131148
4,-4.825788,4.971517,3.280436,1.615437,1.807248,1.378758,1.799174,0.364345,0.185690,-0.081277,...,-0.208892,-0.055993,0.553002,-0.134816,-0.303334,-0.022790,0.207409,0.173916,-0.421973,0.128092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1916,-0.642497,3.278563,-4.471106,1.113510,-2.386641,-1.101542,-1.035676,1.063823,-1.114838,-1.146903,...,0.198759,0.087146,0.264301,-0.111724,0.162525,0.345183,-0.149400,0.249217,-0.218045,0.374411
1917,3.159034,-2.385640,3.341745,7.511678,2.356143,-0.179058,-1.531165,0.226394,-1.157533,-0.547742,...,0.072122,-0.170749,-0.284839,0.106780,0.237520,-0.297730,0.023461,0.337971,0.376953,-0.043910
1918,3.761368,-3.648268,4.604673,1.626326,-0.225079,1.121218,-0.905467,0.435164,-3.618938,-1.131269,...,-0.686761,-0.252695,0.859960,0.436831,0.834351,-0.085112,-0.171217,0.160353,0.070980,-0.017718
1919,17.082366,7.298191,4.599831,-0.426151,-2.051958,1.274194,-1.274484,1.810008,-1.948592,1.720682,...,0.054637,0.166259,-0.051463,0.036512,-0.152514,-0.527832,0.212786,-0.301137,-0.156658,-0.220112


In [28]:
    result = pd.concat([df, rest],axis = 1, join = 'outer', ignore_index=False, sort=False)


In [29]:
result

,0,1,2,3,4,5,6,7,8,9,...,Specific Learning Disorder,Autism Spectrum Disorder,Disruptive,Communication Disorder,Depressive Disorders,Other Disorders,Sex,Age,Study_Site,IDs
0,-3.135464,3.094747,-3.792136,1.046076,-2.982556,-0.760645,-1.073833,-1.305185,0.227560,0.439760,...,0,0,0,0,0,0,1,6.728040,2,NDARAA075AMK
1,-8.735916,-4.395335,5.553377,3.116078,1.681089,-0.786336,-0.584752,-1.174209,1.274068,0.228281,...,0,0,1,0,0,1,0,5.545744,1,NDARAA112DMH
2,1.701273,4.359496,-1.785404,0.665948,1.634100,-0.659093,0.714908,-1.606840,2.279333,0.542853,...,0,0,1,0,0,1,0,7.475929,1,NDARAA117NEJ
3,-8.155979,-5.996179,6.760203,2.807360,2.340080,1.566508,1.399411,0.363007,-1.309300,1.665582,...,1,1,0,0,0,1,0,13.627880,1,NDARAA947ZG5
4,-4.825788,4.971517,3.280436,1.615437,1.807248,1.378758,1.799174,0.364345,0.185690,-0.081277,...,0,0,0,0,0,1,1,7.982660,1,NDARAA948VFH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1916,-0.642497,3.278563,-4.471106,1.113510,-2.386641,-1.101542,-1.035676,1.063823,-1.114838,-1.146903,...,0,0,0,0,0,0,1,11.629363,3,NDARZN148PMN
1917,3.159034,-2.385640,3.341745,7.511678,2.356143,-0.179058,-1.531165,0.226394,-1.157533,-0.547742,...,0,0,0,0,1,0,1,14.878736,1,NDARZN277NR6
1918,3.761368,-3.648268,4.604673,1.626326,-0.225079,1.121218,-0.905467,0.435164,-3.618938,-1.131269,...,0,0,0,0,0,1,1,16.379534,1,NDARZN610GTY
1919,17.082366,7.298191,4.599831,-0.426151,-2.051958,1.274194,-1.274484,1.810008,-1.948592,1.720682,...,0,0,0,0,0,0,1,15.029545,3,NDARZN677EYE


In [30]:
result.to_csv('result.csv')  